In [27]:
# Imports
import datetime, json, pytz
import os
import pandas as pd
from copy import copy
from django.apps import apps as django_apps
from django.db.models.signals import post_save
from django.core.exceptions import ValidationError
from flourish_caregiver.helper_classes import SequentialCohortEnrollment
from flourish_caregiver.helper_classes.schedule_dict import child_schedule_dict, caregiver_schedule_dict
from flourish_child.models import Appointment as ChildAppointment
from edc_appointment.models import Appointment as CaregiverAppointment
from edc_appointment.constants import NEW_APPT
from edc_base.utils import age, get_utcnow
from flourish_caregiver.models import CaregiverOffSchedule
from flourish_caregiver.helper_classes.onschedule_helper import OnScheduleHelper
from edc_visit_schedule.site_visit_schedules import site_visit_schedules
from edc_reference.site import SiteReferenceConfigError

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [28]:
def get_onschedule_obj(subject_identifier, model_cls, schedule_name, field_name):
    try:
        onsch = model_cls.objects.get(
            schedule_name=schedule_name,
            **{f'{field_name}': subject_identifier})
    except model_cls.DoesNotExist:
        pass
    else:
        return onsch

In [29]:
def update_forms_visit(forms, visit=None, new_visit=None, form_type='crf'):
    visit_attr = 'child_visit' if isinstance(visit, ChildVisit) else 'maternal_visit'
    for form in forms:
        kwargs = {f'{visit_attr}': visit,}
        model_cls = django_apps.get_model(form.model)
        if form_type == 'requisition':
            kwargs.update({'panel__name': form.panel_name})
            post_save.disconnect(dispatch_uid='senaite_sample_create_on_post_save')
        try:
            model_obj = model_cls.objects.get(**kwargs)
        except model_cls.DoesNotExist:
            continue
        else:
            print(model_obj)
            if visit_attr == 'child_visit':
                model_obj.child_visit = new_visit
            else:
                model_obj.maternal_visit = new_visit
            try:
                model_obj.save()
            except SiteReferenceConfigError:
                continue

In [30]:
def delete_old_metadata(visit):
    reqs = RequisitionMetadata.objects.filter(
        subject_identifier=visit.subject_identifier,
        visit_code=visit.visit_code,
        schedule_name=visit.schedule_name,)
    reqs.filter(entry_status='keyed').update(entry_status='required')
    reqs.delete()
    
    crfs = CrfMetadata.objects.filter(
        subject_identifier=visit.subject_identifier,
        visit_code=visit.visit_code,
        schedule_name=visit.schedule_name,)
    crfs.filter(entry_status='keyed').update(entry_status='required')
    crfs.delete()

In [31]:
def delete_old_schedule_models(old_onsch, appt_cls, visit_cls):
    ssh = SubjectScheduleHistory.objects.filter(
        subject_identifier=old_onsch.subject_identifier, schedule_name=old_onsch.schedule_name)
    appts = appt_cls.objects.filter(subject_identifier=old_onsch.subject_identifier,
                                    schedule_name=old_onsch.schedule_name)
    visits = visit_cls.objects.filter(appointment__in=appts)
    if ssh.exists():
        ssh.delete()
    for visit in visits:
        visit.delete()
    appts.delete()
    old_onsch.delete()

In [32]:
def enrol_sq_fu(cohort, child_idx, fu_schedule_name, fu_onschedule_model, old_onsch, is_caregiver=False):
    sq_helper_cls = SequentialCohortEnrollment(child_subject_identifier=child_idx)
    child_consent = sq_helper_cls.child_consent_obj
    child_count = str(child_consent.caregiver_visit_count)
    appt_list = ['appt_type', 'appt_status', 'appt_reason', 'comment', 'appt_datetime']
    subject_identifier=child_idx
    appt_cls=ChildAppointment
    visit_cls=ChildVisit

    if is_caregiver:
        subject_identifier=child_consent.subject_consent.subject_identifier
        appt_cls=CaregiverAppointment
        visit_cls=MaternalVisit
        fu_schedule_name=fu_schedule_name[child_count]
    
    sq_helper_cls.enrol_fu_schedule(cohort=cohort,
                                    subject_identifier=subject_identifier,
                                    schedule_name=fu_schedule_name,
                                    onschedule_model=fu_onschedule_model,
                                    is_caregiver=is_caregiver,
                                    onschedule_datetime=old_onsch.onschedule_datetime)
    sq_onsch_cls = django_apps.get_model(fu_onschedule_model)

    try:
        sq_onsch = sq_onsch_cls.objects.get(
            subject_identifier=subject_identifier, schedule_name=fu_schedule_name)
    except sq_onsch_cls.DoesNotExist:
        print('not enroled', subject_identifier, fu_schedule_name)
    else:
        old_appts = appt_cls.objects.filter(
            subject_identifier=subject_identifier, schedule_name=old_onsch.schedule_name,).exclude(
            appt_status=NEW_APPT)
        for appt in old_appts:
            visit_code = ''
            for value in appt.visit_code:
                if not value.isalpha():
                    visit_code += value
            visit_code += cohort.replace('cohort_', '').upper()
            new_appt = appt_cls.objects.filter(
                subject_identifier=subject_identifier, schedule_name=sq_onsch.schedule_name,
                visit_code=visit_code, visit_code_sequence=appt.visit_code_sequence)
            appt_dict = {key: getattr(appt, key) for key in appt_list}
            new_appt.update(**appt_dict)
            try:
                visit = visit_cls.objects.get(appointment=appt)
            except visit_cls.DoesNotExist:
                continue
            else:
                try:
                    new_visit = visit_cls.objects.get(appointment=new_appt[0])
                except visit_cls.DoesNotExist:
                    new_visit = copy(visit)
                    new_visit.pk = None
                    new_visit.visit_code = new_appt[0].visit_code
                    new_visit.appointment = new_appt[0]
                    new_visit.schedule_name = new_appt[0].schedule_name
                    new_visit.visit_schedule_name = new_appt[0].visit_schedule_name
                    new_visit.save()
                        
                crfs = CrfMetadata.objects.filter(
                    subject_identifier=subject_identifier,
                    visit_code=visit.visit_code,
                    visit_code_sequence=visit.visit_code_sequence,
                    schedule_name=visit.schedule_name,
                    entry_status='KEYED')
                print(visit, new_visit)
                update_forms_visit(forms=crfs, visit=visit, new_visit=new_visit)

                reqs = RequisitionMetadata.objects.filter(
                    subject_identifier=subject_identifier,
                    visit_code=visit.visit_code,
                    visit_code_sequence=visit.visit_code_sequence,
                    schedule_name=visit.schedule_name,
                    entry_status='KEYED')
                update_forms_visit(forms=reqs, visit=visit, new_visit=new_visit, form_type='requisition')
                delete_old_metadata(visit)
        delete_old_schedule_models(old_onsch, appt_cls, visit_cls)

In [33]:
# Query participants who have been sequentially enrolled
sq_enrolled = Cohort.objects.values(
    'subject_identifier').annotate(cohort_count=Count('subject_identifier')).filter(cohort_count__gt=1)
child_identifiers = sq_enrolled.values_list('subject_identifier', flat=True)
child_identifiers = set(child_identifiers)

In [34]:
list(child_identifiers)

['B142-040990804-2-10',
 'B142-040990984-2-10',
 'B142-040990166-6-10',
 'B142-040990026-2-10',
 'B142-040990096-5-35',
 'B142-040990814-1-10',
 'B142-040990551-9-10',
 'B142-040990074-2-10',
 'B142-040990359-7-10',
 'B142-040990448-8-10',
 'B142-040990106-2-10',
 'B142-040990081-7-10',
 'B142-040990065-0-35',
 'B142-040990647-5-10',
 'B142-040990854-7-10',
 'B142-040990603-8-10',
 'B142-040990662-4-10',
 'C142-040990953-7-10',
 'B142-040990303-5-10',
 'B142-040991022-0-10',
 'B142-040990851-3-10',
 'B142-040990393-6-10',
 'B142-040990425-6-10',
 'B142-040990302-7-10',
 'B142-040991030-3-10',
 'B142-040990597-2-10',
 'B142-040990491-8-10',
 'B142-040990954-5-10',
 'B142-040990422-3-10',
 'B142-040990528-7-60',
 'B142-040990670-7-10',
 'B142-040990209-4-10',
 'B142-040990243-3-10',
 'B142-040990329-0-10',
 'B142-040990276-3-10',
 'B142-040990187-2-10',
 'B142-040990549-3-10',
 'B142-040990245-8-10',
 'B142-040990330-8-10',
 'B142-040990046-0-10',
 'B142-040990973-5-10',
 'B142-040990104

In [13]:
for child_idx in child_identifiers:
    latest_cohort = Cohort.objects.filter(subject_identifier=child_idx).latest('assign_datetime') 
   
    fu_schedules = CohortSchedules.objects.filter(
        schedule_type='followup',
        cohort_name=latest_cohort.name, )

    for schedule in fu_schedules:
        onschedule_model = schedule.onschedule_model
        model_cls = django_apps.get_model(onschedule_model)
        onsch = None

        if schedule.onschedule_model.startswith('flourish_child'):
            onsch = get_onschedule_obj(child_idx, model_cls, schedule.schedule_name, 'subject_identifier')
            if onsch:
                print(child_idx, onsch.schedule_name)
                child_sq_onsch = child_schedule_dict[latest_cohort.name]['sq_followup']['onschedule_model']
                child_sq_name = child_schedule_dict[latest_cohort.name]['sq_followup']['name']
                enrol_sq_fu(cohort=latest_cohort.name,
                            child_idx=child_idx,
                            fu_schedule_name=child_sq_name,
                            fu_onschedule_model=child_sq_onsch,
                            old_onsch=onsch)
        else:
            onsch = get_onschedule_obj(child_idx, model_cls, schedule.schedule_name, 'child_subject_identifier')
            if onsch:
                print(child_idx, onsch.schedule_name)
                caregiver_sq_onsch = caregiver_schedule_dict[latest_cohort.name]['sq_followup']['onschedule_model']
                caregiver_sq_name = caregiver_schedule_dict[latest_cohort.name]['sq_followup']

                enrol_sq_fu(cohort=latest_cohort.name,
                            child_idx=child_idx,
                            fu_schedule_name=caregiver_sq_name,
                            fu_onschedule_model=caregiver_sq_onsch,
                            old_onsch=onsch,
                            is_caregiver=True)

In [83]:
def enrol_sq_secondary_aims(
    cohort, child_idx, sec_onsch_model, sec_onsch_name, old_onsch, old_schedule, old_sec_obj,
    is_caregiver=False):
    
    base_appt_datetime = None
    sq_helper_cls = SequentialCohortEnrollment(child_subject_identifier=child_idx)
    child_consent = sq_helper_cls.child_consent_obj
    kwargs = {'onschedule_model__startswith': 'flourish_child'}
    subject_identifier=child_idx
    field_name = 'subject_identifier'
    visit_cls = ChildVisit
    appt_cls = ChildAppointment

    if is_caregiver:
        kwargs = {'child_count': old_schedule.child_count, }
        subject_identifier=child_consent.subject_consent.subject_identifier
        field_name = 'child_subject_identifier'
        visit_cls = MaternalVisit
        appt_cls = CaregiverAppointment

    try:
        fu_quarterly = CohortSchedules.objects.get(
            schedule_type='followup_quarterly',
            cohort_name=old_schedule.cohort_name, **kwargs)
    except CohortSchedules.DoesNotExist:
        print('eeerh...', child_idx, old_schedule.cohort_name)
    else:
        quart_onsch_cls = django_apps.get_model(fu_quarterly.onschedule_model)
        try:
            quart_onsch = quart_onsch_cls.objects.get(
                schedule_name=fu_quarterly.schedule_name,
                **{f'{field_name}': child_idx})
        except quart_onsch_cls.DoesNotExist:
            pass
        else:
            complete_appts = appt_cls.objects.filter(
                subject_identifier=subject_identifier, schedule_name=quart_onsch.schedule_name).exclude(
                appt_status='new')
            visit_codes = complete_appts.values_list('visit_code', flat=True)

            visit_objs = visit_cls.objects.filter(
                subject_identifier=subject_identifier, schedule_name=old_onsch.schedule_name)
            if visit_objs.exists():
                base_appt_datetime = visit_objs[0].report_datetime

        onschedule_datetime = old_sec_obj.onschedule_datetime
        base_appt_datetime = base_appt_datetime or onschedule_datetime

        sq_helper_cls.put_on_schedule(sec_onsch_model, sec_onsch_name, subject_identifier,
                                      base_appt_datetime=base_appt_datetime, is_caregiver=is_caregiver,
                                      onschedule_datetime=onschedule_datetime)
        new_codes = []
        for code in visit_codes:
            if code.endswith('M'):
                code = code.replace('M', '')
            code += 'S'
            new_codes.append(code)

        new_appts = appt_cls.objects.filter(
            subject_identifier=subject_identifier, schedule_name=sec_onsch_name, visit_code__in=new_codes)
        if new_appts.exists():
            new_appts.delete()
        delete_old_schedule_models(old_sec_obj, appt_cls, visit_cls)
        print('done', subject_identifier)

In [84]:
count = 0
for child_idx in child_identifiers:
    latest_cohort = Cohort.objects.filter(subject_identifier=child_idx, ).latest('assign_datetime') 

    if '_sec' in latest_cohort.name:
        # Check if previous schedule was FU
        try:
            enrol_cohort = Cohort.objects.exclude(name__icontains='_sec').get(
                subject_identifier=child_idx, enrollment_cohort=True)
        except Cohort.DoesNotExist:
            continue
        else:
            count += 1
            fu_schedules = CohortSchedules.objects.filter(
                schedule_type='followup',
                cohort_name=enrol_cohort.name, )
            for schedule in fu_schedules:
                onschedule_model = schedule.onschedule_model
                model_cls = django_apps.get_model(onschedule_model)
                onsch = None

                if schedule.onschedule_model.startswith('flourish_child'):
                    onsch = get_onschedule_obj(
                        child_idx, model_cls, schedule.schedule_name, 'subject_identifier')

                    old_sec_onsch = child_schedule_dict[latest_cohort.name]['quarterly']['onschedule_model']
                    old_sec_name = child_schedule_dict[latest_cohort.name]['quarterly']['name']
                    old_sec_cls = django_apps.get_model(old_sec_onsch)
                    old_sec_obj = get_onschedule_obj(
                        child_idx, old_sec_cls, old_sec_name, 'subject_identifier')
                    if onsch and old_sec_obj:
                        print(child_idx, onsch.schedule_name)
                        onsch_model = child_schedule_dict[latest_cohort.name]['followup_quarterly']['onschedule_model']
                        onsch_name = child_schedule_dict[latest_cohort.name]['followup_quarterly']['name']

                        enrol_sq_secondary_aims(
                            cohort=latest_cohort.name,
                            child_idx=child_idx,
                            sec_onsch_model=onsch_model,
                            sec_onsch_name=onsch_name,
                            old_onsch=onsch,
                            old_schedule=schedule,
                            old_sec_obj=old_sec_obj, )
                else:
                    onsch = get_onschedule_obj(
                        child_idx, model_cls, schedule.schedule_name, 'child_subject_identifier')

                    old_sec_onsch = caregiver_schedule_dict[latest_cohort.name]['quarterly']['onschedule_model']
                    old_sec_name = caregiver_schedule_dict[latest_cohort.name]['quarterly'][str(schedule.child_count)]
                    old_sec_cls = django_apps.get_model(old_sec_onsch)
                    old_sec_obj = get_onschedule_obj(
                        child_idx, old_sec_cls, old_sec_name, 'child_subject_identifier')
                    if onsch and old_sec_obj:
                        print(child_idx, onsch.schedule_name)
                        onsch_model = caregiver_schedule_dict[latest_cohort.name]['followup_quarterly']['onschedule_model']
                        onsch_name = caregiver_schedule_dict[latest_cohort.name]['followup_quarterly'][str(schedule.child_count)]

                        enrol_sq_secondary_aims(
                            cohort=latest_cohort.name,
                            child_idx=child_idx,
                            sec_onsch_model=onsch_model,
                            sec_onsch_name=onsch_name,
                            old_onsch=onsch,
                            old_schedule=schedule,
                            old_sec_obj=old_sec_obj,
                            is_caregiver=True)
print(count)

B142-040990448-8-10 a_fu1_schedule1
done B142-040990448-8
B142-040990448-8-10 child_a_fu_schedule1
done B142-040990448-8-10
B142-040990543-6-10 a_fu1_schedule1
done B142-040990543-6
B142-040990543-6-10 child_a_fu_schedule1
done B142-040990543-6-10
B142-040990460-3-10 a_fu1_schedule1
done B142-040990460-3
B142-040990460-3-10 child_a_fu_schedule1
done B142-040990460-3-10
B142-040990592-3-10 a_fu1_schedule1
done B142-040990592-3
B142-040990592-3-10 child_a_fu_schedule1
done B142-040990592-3-10
B142-040990489-2-10 a_fu1_schedule1
done B142-040990489-2
B142-040990489-2-10 child_a_fu_schedule1
done B142-040990489-2-10
B142-040990972-7-10 a_fu1_schedule1
done B142-040990972-7
B142-040990972-7-10 child_a_fu_schedule1
done B142-040990972-7-10
B142-040990794-5-10 a_fu1_schedule1
done B142-040990794-5
B142-040990794-5-10 child_a_fu_schedule1
done B142-040990794-5-10
C142-040990956-0-10 a_fu1_schedule1
done C142-040990956-0
C142-040990956-0-10 child_a_fu_schedule1
done C142-040990956-0-10
B142-040